In [1]:
import json

import math
import pandas as pd 
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm.notebook import trange

In [2]:
ori_df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

In [3]:
df = ori_df[['location','total_cases','date']].copy()

In [4]:
df["total_cases"] = df["total_cases"].fillna(0)
df.head

<bound method NDFrame.head of            location  total_cases        date
0       Afghanistan          0.0  2020-01-05
1       Afghanistan          0.0  2020-01-06
2       Afghanistan          0.0  2020-01-07
3       Afghanistan          0.0  2020-01-08
4       Afghanistan          0.0  2020-01-09
...             ...          ...         ...
402698     Zimbabwe     266362.0  2024-05-15
402699     Zimbabwe     266362.0  2024-05-16
402700     Zimbabwe     266362.0  2024-05-17
402701     Zimbabwe     266362.0  2024-05-18
402702     Zimbabwe     266362.0  2024-05-19

[402703 rows x 3 columns]>

In [5]:
asia=df.loc[df['location'] == 'Asia']
africa=df.loc[df['location'] == 'Africa']
europe=df.loc[df['location'] == 'Europe'].reset_index()
north_america=df.loc[df['location'] == 'North America']
oceania=df.loc[df['location'] == 'Oceania']
south_America=df.loc[df['location'] == 'South America']

In [6]:
europe.tail(20)

,index,location,total_cases,date
1589,113207,Europe,252574158.0,2024-05-12
1590,113208,Europe,252574158.0,2024-05-13
1591,113209,Europe,252574158.0,2024-05-14
1592,113210,Europe,252574158.0,2024-05-15
1593,113211,Europe,252574158.0,2024-05-16
1594,113212,Europe,252574158.0,2024-05-17
1595,113213,Europe,252574158.0,2024-05-18
1596,113214,Europe,252591016.0,2024-05-19
1597,113215,Europe,0.0,2024-05-20
1598,113216,Europe,0.0,2024-05-21


In [7]:
new_df=pd.concat([asia,africa,europe,north_america,oceania,south_America]).reset_index().drop(columns='index')

In [8]:
new_df.head()

,level_0,location,total_cases,date
0,19168,Asia,1.0,2020-01-05
1,19169,Asia,1.0,2020-01-06
2,19170,Asia,1.0,2020-01-07
3,19171,Asia,1.0,2020-01-08
4,19172,Asia,1.0,2020-01-09


In [9]:
new_df[new_df['location']=='Europe']

,level_0,location,total_cases,date
3206,0,Europe,1.0,2020-01-05
3207,1,Europe,1.0,2020-01-06
3208,2,Europe,1.0,2020-01-07
3209,3,Europe,1.0,2020-01-08
3210,4,Europe,1.0,2020-01-09
...,...,...,...,...
4810,1604,Europe,0.0,2024-05-27
4811,1605,Europe,0.0,2024-05-28
4812,1606,Europe,0.0,2024-05-29
4813,1607,Europe,0.0,2024-05-30


In [10]:
new_df['date']

0       2020-01-05
1       2020-01-06
2       2020-01-07
3       2020-01-08
4       2020-01-09
           ...    
9612    2024-05-15
9613    2024-05-16
9614    2024-05-17
9615    2024-05-18
9616    2024-05-19
Name: date, Length: 9617, dtype: object

In [11]:
new_df['year']=pd.to_datetime(new_df['date']).dt.year
new_df['month']=pd.to_datetime(new_df['date']).dt.month
new_df['day']=pd.to_datetime(new_df['date']).dt.day

In [12]:
europe['date'] = pd.to_datetime(new_df['date'])
dt_start = pd.to_datetime('2020-01-05')
dt_end = pd.to_datetime('2024-04-28')

europe=europe[europe['date'].between(dt_start, dt_end)]

In [13]:
europe=europe.drop(['index'], axis=1)

In [14]:
europe

,location,total_cases,date
0,Europe,1.0,2020-01-05
1,Europe,1.0,2020-01-06
2,Europe,1.0,2020-01-07
3,Europe,1.0,2020-01-08
4,Europe,1.0,2020-01-09
...,...,...,...
1571,Europe,252521192.0,2024-04-24
1572,Europe,252521192.0,2024-04-25
1573,Europe,252521192.0,2024-04-26
1574,Europe,252521192.0,2024-04-27


## Start predicting next 5 days

In [15]:
# df_test = pd.DataFrame()
# df_test['ds'] = europe['date']
# df_test['y'] = europe['total_cases'].astype(int)
# # df_test.set_index('ds', inplace=True)
# df_test.info()

In [16]:
# from prophet import Prophet
# m = Prophet()
# m.fit(df_test)
# future = m.make_future_dataframe(periods=365)
# future.tail()
# forecast = m.predict(future)

# forecast[['date', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [17]:
# forecast['yhat']=forecast['yhat'].astype('int') 
# forecast.tail()

In [18]:
# forecast.head()

In [19]:
# europe_new = pd.DataFrame()

In [20]:
# europe_new['date']=forecast['ds']
# europe_new['total_case']=forecast['yhat'].astype(int)
# europe_new

In [21]:
# europe_new.insert(0, 'location', 'europe')
# europe_new

In [22]:
europe.to_csv('europe_ph.csv')